In [1]:
import pandas as pd
import numpy as np

In [40]:
from sklearn import preprocessing

In [55]:
from sklearn.model_selection import train_test_split

In [69]:
import re
import string

In [99]:
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
from sklearn.ensemble import RandomForestClassifier

In [106]:
from sklearn.metrics import accuracy_score

In [117]:
from sklearn.naive_bayes import MultinomialNB

In [116]:
import pickle

In [118]:
from sklearn.metrics import confusion_matrix

In [4]:
agora_df = pd.read_csv('/Volumes/SeagateBlue/ds_data/dark_net_market/Agora.csv', encoding = "ISO-8859-1")
agora_df.sample(10)

,Vendor,Category,Item,Item Description,Price,Origin,Destination,Rating,Remarks
29425,rc4me,Drugs/Psychedelics/2C,2C-I - 25g,IUPAC name : 2 5-Dimethoxy-4-iodophenethylamin...,2.796305871090909 BTC,China,NaN,5.00/5,Average price may be skewed outliar > .5 BTC f...
108165,QualityMedicine,Drugs/Cannabis/Weed,AA Indoor Blue Dream HP,Indoor Blue Dream nice frost dense nugs.,2.01817464 BTC,USA,NaN,4.884/5,NaN
90338,TheWhiteShadow,Drugs/Psychedelics/NB,100 x 25i-NBOME 1200ug Blotters,*****SR SELLER - AGORA HALF PRICE STARTING UP ...,0.42301024 BTC,Australia,Australia,[0 deals],NaN
59555,SweetShop,Drugs/Benzos,Flubromazolam Mints .50mg x25,Want a convenient way to get flubromazolam bu...,0.11994839333333333 BTC,USA,NaN,~5/5,NaN
59895,Auxilium,Drugs/Steroids,Equipoise (Boldenone Undecylenate) - 250g Raw ...,Please read entire profile before purchase!!! ...,2.6343427714285714 BTC,USA,World,~5/5,NaN
2048,fake,Information/eBooks,Cloning Credit Cards - 2 ebooks,A combined pre-play and downgrade attack on EM...,0.03352600929203538 BTC,NaN,NaN,4.782/5,NaN
93800,TheHappyGuy,Drugs/Cannabis/Weed,4oz Bubba Trainwreak,Finalize Early Listing: Ships PRIORITY 4oz Un...,2.0137963500000002 BTC,USA,USA,4.993/5,NaN
32186,Hackyboy,Services/Money,Anon. exchanger cash deposit to BTC,**This is for Cash Deposit only at Bank of Ame...,0.0041155716666666665 BTC,Torland,NaN,4.897/5,NaN
27810,pablo-ijscobar,Drugs/Dissociatives/MXE,10g MXE Methoxetamine Free Shipping,Here i offer you MXE (Methoxetamine). This is ...,0.9226989057446806 BTC,Netherlands,Worldwide,4.92/5,NaN
37681,5588,Drugs/Stimulants/Cocaine,14grams half OZ of COCAINE Coke 14G ABOVE 70% ...,Above 70% and that's not bullshit! Over 75% p...,2.052269013913043 BTC,USA,NaN,4.979/5,NaN


In [13]:
# clean the column names
agora_df.columns = agora_df.columns.str.lstrip().str.lower().str.replace(' ', '_')
agora_df.columns

Index(['vendor', 'category', 'item', 'item_description', 'price', 'origin',
       'destination', 'rating', 'remarks'],
      dtype='object')

In [89]:
#remove row with na in item column
agora_df.dropna(subset=['item'], inplace=True)

In [28]:
agora_df.category.value_counts()

Drugs/Cannabis/Weed                                                         21272
Drugs/Ecstasy/Pills                                                          7534
Drugs/Ecstasy/MDMA                                                           6116
Drugs/Stimulants/Cocaine                                                     6007
Drugs/Prescription                                                           5561
Drugs/Benzos                                                                 5393
Drugs/Cannabis/Concentrates                                                  4257
Drugs/Psychedelics/LSD                                                       3775
Drugs/Cannabis/Hash                                                          3241
Drugs/Steroids                                                               2779
Drugs/Stimulants/Meth                                                        2467
Drugs/Stimulants/Speed                                                       2401
Drugs/RCs       

In [27]:
agora_df.shape

(109689, 9)

In [26]:
agora_df.loc[agora_df.category.isin(
    agora_df.category.value_counts()[agora_df.category.value_counts() > 1].index)].shape

(109684, 9)

In [29]:
# remove the rows that have a category value count of 1
agora_df = agora_df.loc[agora_df.category.isin(
    agora_df.category.value_counts()[agora_df.category.value_counts() > 1].index)]

In [31]:
agora_df.category.value_counts().shape

(104,)

In [42]:
le = preprocessing.LabelEncoder()

In [95]:
le.fit(agora_df.category)

LabelEncoder()

In [96]:
le.classes_

array(['Chemicals', 'Counterfeits/Accessories', 'Counterfeits/Clothing',
       'Counterfeits/Electronics', 'Counterfeits/Money',
       'Counterfeits/Watches', 'Data/Accounts', 'Data/Pirated',
       'Data/Software', 'Drug paraphernalia/Containers',
       'Drug paraphernalia/Grinders',
       'Drug paraphernalia/Injecting equipment/Filters',
       'Drug paraphernalia/Injecting equipment/Needles',
       'Drug paraphernalia/Injecting equipment/Other',
       'Drug paraphernalia/Injecting equipment/Syringes',
       'Drug paraphernalia/Paper', 'Drug paraphernalia/Pipes',
       'Drug paraphernalia/Scales', 'Drug paraphernalia/Stashes',
       'Drugs/Barbiturates', 'Drugs/Benzos',
       'Drugs/Cannabis/Concentrates', 'Drugs/Cannabis/Edibles',
       'Drugs/Cannabis/Hash', 'Drugs/Cannabis/Seeds',
       'Drugs/Cannabis/Shake/trim', 'Drugs/Cannabis/Synthetics',
       'Drugs/Cannabis/Weed', 'Drugs/Dissociatives/GBL',
       'Drugs/Dissociatives/GHB', 'Drugs/Dissociatives/Ketamine',
    

In [97]:
y = le.transform(agora_df.category)
y[:5]

array([93, 93, 93, 93, 93])

In [79]:
# text preprocessing
alphanumeric = lambda x: re.sub(r'\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub(r'[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [92]:
agora_df.item = agora_df.item.map(alphanumeric).map(punc_lower)
agora_df.item.head()

0                             month huluplus gift code
1    pay tv sky uk  sky germany hd tv  and much mor...
2                 official account creator extreme    
3                            vpn   tor   sock tutorial
4                               facebook hacking guide
Name: item, dtype: object

In [93]:
X = agora_df.item.values
X[:5]

array(['  month huluplus gift code',
       'pay tv sky uk  sky germany hd tv  and much more   cccam service   months',
       'official account creator extreme    ',
       'vpn   tor   sock tutorial', 'facebook hacking guide'],
      dtype=object)

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [100]:
cv1 = CountVectorizer(stop_words='english')

X_train_cv1 = cv1.fit_transform(X_train)
X_test_cv1  = cv1.transform(X_test)

In [105]:
rfc = RandomForestClassifier()

In [107]:
rfc.fit(X_train_cv1, y_train)

/Users/jonathandawson/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [108]:
# The accuracy
y_pred = rfc.predict(X_test_cv1)
print(accuracy_score(y_test, y_pred))

0.8388826082199774


In [109]:
print(y_pred.shape)
accuracy_score(y_test, y_pred, normalize=False)

(27421,)


23003

In [110]:
rfc.score(X_test_cv1, y_test)

0.8388826082199774

In [112]:
nb = MultinomialNB()

In [113]:
nb.fit(X_train_cv1, y_train)
nb.score(X_test_cv1, y_test)

0.7846540972247548

In [127]:
np.set_printoptions(threshold=1000)
#np.set_printoptions(threshold=np.inf)

In [128]:
confusion_matrix(y_pred, y_test)

array([[12,  0,  1, ...,  0,  0,  0],
       [ 0, 41,  4, ...,  0,  0,  0],
       [ 0,  2, 67, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 45,  0,  0],
       [ 0,  0,  0, ...,  0, 17,  3],
       [ 0,  0,  0, ...,  0,  3,  8]])

In [129]:
with open('cv1.pickle', 'wb') as f:
    pickle.dump(cv1, f, pickle.HIGHEST_PROTOCOL)

In [131]:
with open('nb.pickle', 'wb') as f:
    pickle.dump(nb, f, pickle.HIGHEST_PROTOCOL)

In [132]:
with open('le.pickle', 'wb') as f:
    pickle.dump(le, f, pickle.HIGHEST_PROTOCOL)